In [1]:
import numpy as np
part1 = np.load("part1a.npz")
part1.files
Ic_0 = part1["Ic_0"]
N = part1["N"]
gamma = part1["gamma"]
Lc = part1["Lc"]
Svc_0 = part1["Svc_0_pmf"]

In [2]:
import pandas as pd
S_0 = N-(Ic_0.sum())
I_0 = Ic_0.sum()
R_0 = 0
data = pd.DataFrame(Svc_0)

In [3]:
import random 
from random import Random
random.seed(a=None, version=2)
B0c = []
B1c = []
B2c = []
B3c = []
for i in range(4):
    B0c.append(random.uniform(0, 0.25))
    B1c.append(random.uniform(0.25, 0.5))
    B2c.append(random.uniform(0.5, 0.75))
    B3c.append(random.uniform(0.75, 1.0))

In [4]:
Bvc = np.array([B0c, B1c, B2c, B3c])
for i in range(4):
    Bvc[i].sort()
Bvc = pd.DataFrame(Bvc)
Bvc

,0,1,2,3
0,0.005883,0.020322,0.027975,0.219545
1,0.262807,0.326584,0.408069,0.474912
2,0.507419,0.513248,0.583773,0.692667
3,0.750569,0.769840,0.830690,0.907886


In [5]:
def S_rate(Svc, Ic, N, Bvc, gamma):
    """Calculate the rate of change of suceptible group split by vulnerability and comorbidity, return a
    tuple where the first element is split on both categories, and the second element only split by comorbidity"""
    #This makes sure that once population hits 0 it doesn't affect the differential
    Svc[Svc < 0] = 0
    dSvcdt = ((Bvc * Svc * Ic)*-1)/N
    return dSvcdt

In [6]:
def I_rate(dSvcdt, Ic, gamma):
    """This function takes in the rate of change of S, sums along the vulnerability axis, and subtracts 
    Ic*gamma"""
    Ic[Ic < 0]= 0
    dIcdt = -dSvcdt.sum() - gamma*Ic
    return dIcdt

In [7]:
def R_rate(Ic, gamma):
    Ic[Ic < 0]= 0
    dRdt = gamma * Ic.sum()
    return dRdt

In [8]:
def SIR_rates(Svc, Ic, N, Bvc, gamma):
    dSvcdt = S_rate(Svc, Ic, N, Bvc, gamma)
    dIcdt = I_rate(dSvcdt, Ic, gamma)
    dRdt = R_rate(Ic, gamma)
    return dSvcdt, dIcdt, dRdt

In [9]:
#The Svc data multiplied by the population to get raw population values split in both dimensions
Svc = pd.DataFrame(Svc_0) * (N - I_0)
Ic = Ic_0
I = I_0
S = Svc.sum().sum()
R = 0
simulation = [[S, I, R]]
for i in range(120):
    rates = SIR_rates(Svc, Ic, N, Bvc, gamma)
    Svc = Svc + rates[0]
    S = Svc.sum().sum()
    Ic = Ic + rates[1]
    I = Ic.sum()
    R = R + rates[2]
    simulation.append([S, I, R])
results = pd.DataFrame(simulation, columns = ["S", "I", "R"])

In [10]:
results["Total"] = results.sum(axis=1)
results

,S,I,R,Total
0,99747.000000,253.000000,0.000000,100000.0
1,99721.182185,260.746387,18.071429,100000.0
2,99694.335206,268.968624,36.696170,100000.0
3,99666.392601,277.699183,55.908215,100000.0
4,99637.283164,286.972965,75.743871,100000.0
5,99606.930595,296.827465,96.241940,100000.0
6,99575.253137,307.302961,117.443902,100000.0
7,99542.163184,318.442703,139.394113,100000.0
8,99507.566870,330.293109,162.140021,100000.0
9,99471.363632,342.903982,185.732386,100000.0
